# 1396. Design Underground System

Level: Medium

An underground railway system is keeping track of customer travel times between different stations. They are using this data to calculate the average time it takes to travel from one station to another.

Implement the UndergroundSystem class:

    ```void checkIn(int id, string stationName, int t)
        A customer with a card ID equal to id, checks in at the station stationName at time t.
        A customer can only be checked into one place at a time.
    ```

    ```void checkOut(int id, string stationName, int t)
        A customer with a card ID equal to id, checks out from the station stationName at time t.
    ```
    ```double getAverageTime(string startStation, string endStation)
        Returns the average time it takes to travel from startStation to endStation.
        The average time is computed from all the previous traveling times from startStation to endStation that happened directly, meaning a check in at startStation followed by a check out from endStation.
        The time it takes to travel from startStation to endStation may be different from the time it takes to travel from endStation to startStation.
        There will be at least one customer that has traveled from startStation to endStation before getAverageTime is called.
    ```
You may assume all calls to the checkIn and checkOut methods are consistent. If a customer checks in at time t1 then checks out at time t2, then t1 < t2. All events happen in chronological order.

## Solution1:

Store the station name and the time everytime a customer checks in. A tuple `(station, time)` is added to the dictionary `checkins`.
At every checkout, we add the duration of the trip to the dictionary `time_records`.

In [1]:
from collections import defaultdict
import statistics

class UndergroundSystem1:

    def __init__(self):
        self.time_records = defaultdict(list)
        self.checkins = dict()
        # self.n_customers = 0

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.checkins[id] = (stationName, t)

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        s1, t1 = self.checkins[id]
        key = f'{s1}_{stationName}'
        self.time_records[key].append(t-t1)
        
    def getAverageTime(self, startStation: str, endStation: str) -> float:
        key = f'{startStation}_{endStation}'
        return statistics.mean(self.time_records[key])


# Your UndergroundSystem object will be instantiated and called as such:
# obj = UndergroundSystem()
# obj.checkIn(id,stationName,t)
# obj.checkOut(id,stationName,t)
# param_3 = obj.getAverageTime(startStation,endStation)

## Solution 2: Computing a running-average at each checkout.
Instead of storing the durations of each trip, we compute the average and store it with the frequency a trip has been taken in the dictionary `time_records`. This step reduced both computation time and storage.

Removing the entry of a customer at the time of checkout (`self.checkins.pop(id)`) further reduced the memory consumption. 


In [ ]:
class UndergroundSystem2:

    def __init__(self):
        self.time_records = dict()
        self.checkins = dict()

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.checkins[id] = (stationName, t)

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        s1, t1 = self.checkins[id]
        t_diff = t-t1
        key = f'{s1}_{stationName}'
        if key in self.time_records:
            avg, nc = self.time_records[key]
            avg += (t_diff-avg)/(nc+1)
            self.time_records[key] = (avg, nc+1)
        else:
            self.time_records[key] = (t_diff, 1)
        self.checkins.pop(id)
        
        
    def getAverageTime(self, startStation: str, endStation: str) -> float:
        key = f'{startStation}_{endStation}'
        return self.time_records[key][0]